_unchecked_

# CNTK 303: LSTM 网络的深层结构语义建模

DSSM 代表了深层的结构语义模型, 或者更一般的、深层语义相似性模型。DSSM, 由 MSR 深学习技术中心 (DLTC) 开发, 是一种深神经网络 (DNN) 建模技术, 用于表示连续语义空间中的文本字符串 (句子、查询、谓词、实体提及等), 并建模语义两个文本字符串 (例如, Sent2Vec) 的相似性。DSSM 有广泛的应用, 包括信息检索和网页搜索排名 ([黄 et al. 2013](https://www.microsoft.com/en-us/research/publication/learning-deep-structured-semantic-models-for-web-search-using-clickthrough-data/);[沈 et al. 2014a](https://www.microsoft.com/en-us/research/publication/learning-semantic-representations-using-convolutional-neural-networks-for-web-search/),[2014b](https://www.microsoft.com/en-us/research/publication/a-latent-semantic-model-with-convolutional-pooling-structure-for-information-retrieval/)), ad 选择/相关性, 上下文实体搜索和兴趣任务 ([高 et al. 2014a](https://www.microsoft.com/en-us/research/publication/modeling-interestingness-with-deep-neural-networks/), 问答 ([亿等 al., 2014](https://www.microsoft.com/en-us/research/publication/semantic-parsing-for-single-relation-question-answering/)), 图像字幕 ([方 et al., 2014](https://arxiv.org/abs/1411.4952)), 和机器翻译 ([高 et al., 2014b](https://www.microsoft.com/en-us/research/publication/learning-continuous-phrase-representations-for-translation-modeling/)) 等。

DSSM 可用于开发不同类型的项目实体 (例如, 查询和文档) 的潜在语义模型, 以用于各种机器学习任务 (如排序和分类) 的公共低维语义空间。例如, 在 web 搜索排名中, 给定查询的文档的相关性可以很容易地计算为在该空间中它们之间的距离。从 Nvidia 的最新 gpu, 我们可以训练我们的模型上亿个字。对文本处理的深入学习感兴趣的读者可以通过[他等 al., 2014](https://www.microsoft.com/en-us/research/publication/deep-learning-for-natural-language-processing-theory-and-practice-tutorial/)来参考教程。
我们发布了 DSSM (又名 Sent2Vec) 的预测因子和训练有素的模型文件。

## 目标

为了开发这样的机制, 如果给定了一对文档的查询和一组网页文档, 该模型将把输入映射到一个连续的、低维空间中的一对特征向量, 其中一个可以比较文本字符串之间的语义相似性在该空间中使用它们的向量之间的余弦相似性。

![](http://kubicode.me/img/Study-With-Deep-Structured-Semantic-Model/dssm_arch.png)

在上面的图中, 你可以看到如何给定一个查询 ($ Q $) 和一组文档 ($ D_1, D_2, \ldots, D_n $), 你可以生成潜在的表示, 又称语义特征, 然后可以用来生成成对距离度量。度量值可用于排序。

_unchecked_

在上面的图片中, 人们可以看到查询和文档都映射到一个术语向量。虽然基于[word 包](https://en.wikipedia.org/wiki/Bag-of-words_model)的建模是在构建 NLP 模型时需要的第一步, 但它们在单词之间捕获相对位置的能力受到限制。基于卷积的或基于重复的模型的性能更好, 因为它们具有利用单词位置的内在能力。在本教程中, 我们将使用一个简单的说明模型, 使用 LSTM 在[Palangi et. al.](https://www.microsoft.com/en-us/research/wp-content/uploads/2017/02/LSTM_DSSM_IEEE_TASLP.pdf)完成的工作之后对术语向量进行编码。

在本教程中, 我们将向您展示如何构建这样一个网络。 我们使用问答语料库中的一个小样本。此外, 我们将使用一个经常性的网络来开发语义模型, 因为它允许将位置信息与单词标记结合在一起。

**注意**: 数据集非常小, 本教程的重点是演示如何为 DSSM 网络创建最终的建模工作流, 而不是我们能够在这个小数据集上获得的特定数值性能。

In [1]:
# Import the relevant libraries
import math
import numpy as np
import os
from __future__ import print_function # Use a function definition from future version (say 3.x from 2.7 interpreter)

import cntk as C
import cntk.tests.test_utils
cntk.tests.test_utils.set_device_from_pytest_env() # (only needed for our build system)
C.cntk_py.set_fixed_random_seed(1) # fix a random seed for CNTK components

_unchecked_

## 数据准备

### 下载

我们使用问答数据集的抽样来说明如何建模 DSSM 网络。数据集由一对具有[问题和答案](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/ACL15-STAGG.pdf)的句子组成。在本教程中, 我们将数据预处理为两个部分:-词汇文件: 1 文件每一个问题和答案。有1204和1019单词的问题和答案的词汇, 分别。
-QA 文件: 1 文件每个文件的培训和验证数据 (保持), 其中每一个档案转换为[周大福格式](https://cntk.ai/pythondocs/CNTK_202_Language_Understanding.html)。训练和验证文件分别有3500和409句对。

注: 本文作者提供了一小部分原始数据, 用于创建范例网络以供说明之用。

In [2]:
location = os.path.normpath('data/DSSM')
data = {
  'train': { 'file': 'train.pair.tok.ctf' },
  'val':{ 'file': 'valid.pair.tok.ctf' },
  'query': { 'file': 'vocab_Q.wl' },
  'answer': { 'file': 'vocab_A.wl' }
}

import requests

def download(url, filename):
    """ utility function to download a file """
    response = requests.get(url, stream=True)
    with open(filename, "wb") as handle:
        for data in response.iter_content():
            handle.write(data)

if not os.path.exists(location):
    os.mkdir(location)
     
for item in data.values():
    path = os.path.normpath(os.path.join(location, item['file']))

    if os.path.exists(path):
        print("Reusing locally cached:", path)
        
    else:
        print("Starting download:", item['file'])
        url = "http://www.cntk.ai/jup/dat/DSSM/%s.csv"%(item['file'])
        print(url)
        download(url, path)
        print("Download completed")
    item['file'] = path

Starting download: vocab_A.wl
http://www.cntk.ai/jup/dat/DSSM/vocab_A.wl.csv
Download completed
Starting download: train.pair.tok.ctf
http://www.cntk.ai/jup/dat/DSSM/train.pair.tok.ctf.csv
Download completed
Starting download: valid.pair.tok.ctf
http://www.cntk.ai/jup/dat/DSSM/valid.pair.tok.ctf.csv
Download completed
Starting download: vocab_Q.wl
http://www.cntk.ai/jup/dat/DSSM/vocab_Q.wl.csv
Download completed


_unchecked_

### 读者

我们将使用反来读取输入数据。但是, 可以编写自己的读取器或使用 numpy 数组为 CNTK 建模工作流提供数据。您可能希望使用文本编辑器打开 "周大福文件" 来分析输入。注意, 周大福反具有跨多个磁盘的生产规模数据大小的扩展能力。读者还可以用一个简单的标志来抽象出大规模的随机化, 为程序员增加了方便和节省时间。

In [3]:
# Define the vocabulary size (QRY-stands for question and ANS stands for answer)
QRY_SIZE = 1204
ANS_SIZE = 1019

def create_reader(path, is_training):
    return C.io.MinibatchSource(C.io.CTFDeserializer(path, C.io.StreamDefs(
         query = C.io.StreamDef(field='S0', shape=QRY_SIZE,  is_sparse=True),
         answer  = C.io.StreamDef(field='S1', shape=ANS_SIZE, is_sparse=True)
     )), randomize=is_training, max_sweeps = C.io.INFINITELY_REPEAT if is_training else 1)

In [4]:
train_file = data['train']['file']
print(train_file)

if os.path.exists(train_file):
    train_source = create_reader(train_file, is_training=True)
else:
    raise ValueError("Cannot locate file {0} in current directory {1}".format(train_file, os.getcwd()))

validation_file = data['val']['file']
print(validation_file)
if os.path.exists(validation_file):
    val_source = create_reader(validation_file, is_training=False)
else:
    raise ValueError("Cannot locate file {0} in current directory {1}".format(validation_file, os.getcwd()))

data\DSSM\train.pair.tok.ctf
data\DSSM\valid.pair.tok.ctf


_unchecked_

## 模型创建

所提出的 LSTM-RNN 模型依次取句中的每个单词, 提取其信息, 并将其嵌入到一个语义向量中。由于它能够捕捉长期记忆, LSTM-RNN 积累了越来越丰富的信息, 因为它通过这个句子, 当它到达最后一个词, 网络的隐藏层提供了整个句子的语义表示。然后将 `last` block 投影到 `query_vector` 空间, 也称为上面图中的语义特征。

    `                                                                    "query vector"
                                                                              ^
                                                                              |
                                                                          +-------+  
                                                                          | Dense |  
                                                                          +-------+  
                                                                              ^         
                                                                              |         
                                                                         +---------+  
                                                                         | Dropout |  
                                                                         +---------+
                                                                              ^
                                                                              |         
                                                                          +-------+  
                                                                          | Dense |  
                                                                          +-------+  
                                                                              ^         
                                                                              |         
                                                                          +------+   
                                                                          | last |  
                                                                          +------+  
                                                                              ^  
                                                                              |         
                              +------+   +------+   +------+   +------+   +------+   
                         0 -->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |-->| LSTM |
                              +------+   +------+   +------+   +------+   +------+   
                                  ^          ^          ^          ^          ^
                                  |          |          |          |          |
                              +-------+  +-------+  +-------+  +-------+  +-------+
                              | Embed |  | Embed |  | Embed |  | Embed |  | Embed | 
                              +-------+  +-------+  +-------+  +-------+  +-------+
                                  ^          ^          ^          ^          ^
                                  |          |          |          |          |
                    query  ------>+--------->+--------->+--------->+--------->+
    `

同样, 我们可以将应答句投射到 `answer_vector` 。但是, 在我们创建模型之前。让我们为我们的模型定义输入变量。注意, 有一个查询, 并与它配对有一个答案。考虑到这两个词, 我们定义了一系列的单词

In [5]:
# Create the containers for input feature (x) and the label (y)
qry = C.sequence.input_variable(QRY_SIZE)
ans = C.sequence.input_variable(ANS_SIZE)

_unchecked_

**注意**: 您是否闻到上述声明的任何问题。如果你想看看会发生什么, 如果你去与上述声明, 请注释出下面的4语句, 并运行模型。你会发现你的模型抛出一个异常。该异常的详细信息将在[此处](https://cntk.ai/pythondocs/Manual_How_to_debug.html#Runtime-errors)说明。

CNTK 中的每个序列都与表示序列中的单词数的动态轴关联。直观地, 当你有不同大小和词汇的序列时, 它们都需要有自己的动态轴。通过使用命名轴声明输入数据容器来促进这一点。严格地说, 你可以只命名一个, 另一个将是一个默认的动态轴。但是, 为了清楚起见, 我们分别命名两个轴。

In [6]:
# Create the containers for input feature (x) and the label (y)
axis_qry = C.Axis.new_unique_dynamic_axis('axis_qry')
qry = C.sequence.input_variable(QRY_SIZE, sequence_axis=axis_qry)

axis_ans = C.Axis.new_unique_dynamic_axis('axis_ans')
ans = C.sequence.input_variable(ANS_SIZE, sequence_axis=axis_ans)

_unchecked_

在我们可以创建模型之前, 我们需要指定一些与网络体系结构相关的参数。

In [7]:
EMB_DIM   = 25 # Embedding dimension
HIDDEN_DIM = 50 # LSTM dimension
DSSM_DIM = 25 # Dense layer dimension  
NEGATIVE_SAMPLES = 5
DROPOUT_RATIO = 0.2

In [8]:
def create_model(qry, ans):
    with C.layers.default_options(initial_state=0.1):
        qry_vector = C.layers.Sequential([
            C.layers.Embedding(EMB_DIM, name='embed'),
            C.layers.Recurrence(C.layers.LSTM(HIDDEN_DIM), go_backwards=False),
            C.sequence.last,
            C.layers.Dense(DSSM_DIM, activation=C.relu, name='q_proj'),
            C.layers.Dropout(DROPOUT_RATIO, name='dropout qdo1'),
            C.layers.Dense(DSSM_DIM, activation=C.tanh, name='q_enc')
        ])
        
        ans_vector = C.layers.Sequential([
            C.layers.Embedding(EMB_DIM, name='embed'),
            C.layers.Recurrence(C.layers.LSTM(HIDDEN_DIM), go_backwards=False),
            C.sequence.last,
            C.layers.Dense(DSSM_DIM, activation=C.relu, name='a_proj'),
            C.layers.Dropout(DROPOUT_RATIO, name='dropout ado1'),
            C.layers.Dense(DSSM_DIM, activation=C.tanh, name='a_enc')
        ])

    return {
        'query_vector': qry_vector(qry),
        'answer_vector': ans_vector(ans)
    }

# Create the model and store reference in `network` dictionary
network = create_model(qry, ans)

network['query'], network['axis_qry'] = qry, axis_qry
network['answer'], network['axis_ans'] = ans, axis_ans

_unchecked_

## 培训

现在我们已经创建了一个网络, 下一步是找到一个合适的损失函数, 如果 `question` 与正确的配对 `answer` , 则损失将是 0, 否则将是1。换言之, 这一损失应最大限度的相似性 (点产品) 之间的答案向量, 似乎接近回答向量和最小的相似性之间的答案和问题的向量, 不回答对方。

DSSM 的用例通常出现在信息检索中, 对于给定的查询或问题, 在一个贫穷或非答案的海洋中很少有答案。在这种情况下, 输入数据是一对查询和应答 (文档或广告), 它吸引了单击。一个经典的方式来训练将是一个二进制分类器预测点击/不点击 (或等效的2类分类器-一个类, 每个点击或不点击)。可以生成对查询和不正确的答案 (如无单击数据)。但是, 模拟无单击数据的一种方法是对 minibatch 中的其他查询使用查询和答案。这是 `cosine_distance_with_negative_samples` 函数背后的概念。注意: 此函数返回1以更正问题和应答对, 0 为不正确, 这称为*相似性*。因此, 我们使用 1- `cosine_distance_with_negative_samples` 作为我们的损失函数。

In [9]:
def create_loss(vector_a, vector_b):
    qry_ans_similarity = C.cosine_distance_with_negative_samples(vector_a, \
                                                                 vector_b, \
                                                                 shift=1, \
                                                                 num_negative_samples=5)
    return 1 - qry_ans_similarity

In [10]:
# Model parameters
MAX_EPOCHS = 5
EPOCH_SIZE = 10000
MINIBATCH_SIZE = 50

In [11]:
# Create trainer
def create_trainer(reader, network):
    
    # Setup the progress updater
    progress_writer = C.logging.ProgressPrinter(tag='Training', num_epochs=MAX_EPOCHS)

    # Set learning parameters
    lr_per_sample     = [0.0015625]*20 + \
                        [0.00046875]*20 + \
                        [0.00015625]*20 + \
                        [0.000046875]*10 + \
                        [0.000015625]
    lr_schedule       = C.learning_parameter_schedule_per_sample(lr_per_sample, \
                                                 epoch_size=EPOCH_SIZE)
    mms               = [0]*20 + [0.9200444146293233]*20 + [0.9591894571091382]
    mm_schedule       = C.learners.momentum_schedule(mms, \
                                                     epoch_size=EPOCH_SIZE, \
                                                     minibatch_size=MINIBATCH_SIZE)
    l2_reg_weight     = 0.0002

    model = C.combine(network['query_vector'], network['answer_vector'])

    #Notify the network that the two dynamic axes are indeed same
    query_reconciled = C.reconcile_dynamic_axes(network['query_vector'], network['answer_vector'])
  
    network['loss'] = create_loss(query_reconciled, network['answer_vector'])
    network['error'] = None

    print('Using momentum sgd with no l2')
    dssm_learner = C.learners.momentum_sgd(model.parameters, lr_schedule, mm_schedule)

    network['learner'] = dssm_learner
 
    print('Using local learner')
    # Create trainer
    return C.Trainer(model, (network['loss'], network['error']), network['learner'], progress_writer)    

In [12]:
# Instantiate the trainer
trainer = create_trainer(train_source, network)

Using momentum sgd with no l2
Using local learner


In [13]:
# Train 
def do_train(network, trainer, train_source):
    # define mapping from intput streams to network inputs
    input_map = {
        network['query']: train_source.streams.query,
        network['answer']: train_source.streams.answer
        } 

    t = 0
    for epoch in range(MAX_EPOCHS):         # loop over epochs
        epoch_end = (epoch+1) * EPOCH_SIZE
        while t < epoch_end:                # loop over minibatches on the epoch
            data = train_source.next_minibatch(MINIBATCH_SIZE, input_map= input_map)  # fetch minibatch
            trainer.train_minibatch(data)               # update model with it
            t += MINIBATCH_SIZE

        trainer.summarize_training_progress()

In [14]:
do_train(network, trainer, train_source)

Learning rate per 1 samples: 0.0015625
Momentum per 1 samples: 0.0
Finished Epoch[1 of 5]: [Training] loss = 0.343046 * 1522, metric = 0.00% * 1522 5.720s (266.1 samples/s);
Finished Epoch[2 of 5]: [Training] loss = 0.102804 * 1530, metric = 0.00% * 1530 3.464s (441.7 samples/s);
Finished Epoch[3 of 5]: [Training] loss = 0.066461 * 1525, metric = 0.00% * 1525 3.402s (448.3 samples/s);
Finished Epoch[4 of 5]: [Training] loss = 0.048511 * 1534, metric = 0.00% * 1534 3.390s (452.5 samples/s);
Finished Epoch[5 of 5]: [Training] loss = 0.035384 * 1510, metric = 0.00% * 1510 3.383s (446.3 samples/s);


_unchecked_

## 验证

一旦模型被训练, 我们要选择一个模型, 它与验证 ("保持" 集) 具有类似的错误, 作为训练集的错误。

**建议的活动**: 更改世纪数并检查培训和验证错误。

然后, 选择的模型将用于预测。

In [15]:
# Validate
def do_validate(network, val_source):
    # process minibatches and perform evaluation
    progress_printer = C.logging.ProgressPrinter(tag='Evaluation', num_epochs=0)

    val_map = {
        network['query']: val_source.streams.query,
        network['answer']: val_source.streams.answer
        } 

    evaluator = C.eval.Evaluator(network['loss'], progress_printer)

    while True:
        minibatch_size = 100
        data = val_source.next_minibatch(minibatch_size, input_map=val_map)
        if not data:                                 # until we hit the end
            break

        evaluator.test_minibatch(data)

    evaluator.summarize_test_progress()

In [16]:
do_validate(network, val_source)

Finished Evaluation [1]: Minibatch[1-35]: metric = 0.02% * 410;


_unchecked_

## 预测

现在, 我们将创建一个向量表示的查询和答案。然后计算两个向量之间的余弦相似度。当答案接近这个问题时, 你会得到一个很高的相似性, 而一个不正确/部分相关的问题/答案对会导致更小的相似性。这些分数通常用于对 web 文档进行排序以响应查询。

In [17]:
# load dictionaries
query_wl = [line.rstrip('\n') for line in open(data['query']['file'])]
answers_wl = [line.rstrip('\n') for line in open(data['answer']['file'])]
query_dict = {query_wl[i]:i for i in range(len(query_wl))}
answers_dict = {answers_wl[i]:i for i in range(len(answers_wl))}

# let's run a sequence through
qry = 'BOS what contribution did  e1  made to science in 1665 EOS'
ans = 'BOS book author book_editions_published EOS'
ans_poor = 'BOS language human_language main_country EOS'

qry_idx = [query_dict[w+' '] for w in qry.split()] # convert to query word indices
print('Query Indices:', qry_idx)

ans_idx = [answers_dict[w+' '] for w in ans.split()] # convert to answer word indices
print('Answer Indices:', ans_idx)

ans_poor_idx = [answers_dict[w+' '] for w in ans_poor.split()] # convert to fake answer word indices
print('Poor Answer Indices:', ans_poor_idx)

Query Indices: [1202, 1154, 267, 321, 357, 648, 1070, 905, 549, 6, 1203]
Answer Indices: [1017, 135, 91, 137, 1018]
Poor Answer Indices: [1017, 501, 452, 533, 1018]


_unchecked_

将查询、答案和假答案转换为一个热表示。这是一个必要的步骤, 因为输入到我们训练有素的网络接受一热编码输入。

In [18]:
# Create the one hot representations
qry_onehot = np.zeros([len(qry_idx),len(query_dict)], np.float32)
for t in range(len(qry_idx)):
    qry_onehot[t,qry_idx[t]] = 1
    
ans_onehot = np.zeros([len(ans_idx),len(answers_dict)], np.float32)
for t in range(len(ans_idx)):
    ans_onehot[t,ans_idx[t]] = 1
    
ans_poor_onehot = np.zeros([len(ans_poor_idx),len(answers_dict)], np.float32)
for t in range(len(ans_poor_idx)):
    ans_poor_onehot[t, ans_poor_idx[t]] = 1

_unchecked_

对于每个查询和答案一热编码输入, 创建嵌入。注意: 我们将答案嵌入到正确答案和拙劣答案中。我们计算了查询和应答对之间的余弦相似性。余弦相似性的相对值与更高的值表示更好的答案。

In [19]:
qry_embedding = network['query_vector'].eval([qry_onehot])
ans_embedding = network['answer_vector'].eval([ans_onehot])
ans_poor_embedding = network['answer_vector'].eval([ans_poor_onehot])

from scipy.spatial.distance import cosine

print('Query to Answer similarity:', 1-cosine(qry_embedding, ans_embedding))
print('Query to poor-answer similarity:', 1-cosine(qry_embedding, ans_poor_embedding))

Query to Answer similarity: 0.99995367043
Query to poor-answer similarity: 0.999941420215
